In [2]:
import requests
import urllib
import urllib3
import pandas as pd
import json

In [10]:
# https://stackoverflow.com/questions/58350063/obtain-elevation-from-latitude-longitude-coordinates-with-a-simple-python-script

def make_remote_request(url: str, params: dict) -> json:
        """
        Makes the remote request
        Continues making attempts until it succeeds
        """

        count = 1
        while True:
            try:
                response = requests.get((url + urllib.parse.urlencode(params)))
            except (OSError, urllib3.exceptions.ProtocolError) as error:
                print('\n')
                print('*' * 20, 'Error Occured', '*' * 20)
                print(f'Number of tries: {count}')
                print(f'URL: {url}')
                print(error)
                print('\n')
                count += 1
                continue
            break

        return response

    
def elevation_function(x) -> float:
        '''
        Assumes x contain valid latitude and longitude coordinate.
        :param x: x[0] - latitude, x[1] - latitude.
        :return: The elevation at the given coordinates.
        '''
        url = 'https://nationalmap.gov/epqs/pqs.php?'
        params = {'x': x[1],
                  'y': x[0],
                  'units': 'Meters',
                  'output': 'json'}
        result = make_remote_request(url, params)
        return result.json()['USGS_Elevation_Point_Query_Service']['Elevation_Query']['Elevation']

In [ ]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('FPA_FOD_20170508.sqlite')
df = pd.read_sql_query("SELECT * from Fires", conn)

In [14]:
df[["LATITUDE", "LONGITUDE"]][:5].apply(elevation_function, axis=1)

0     924.73
1    1831.26
2    1048.65
3    2361.86
4    2309.84
dtype: float64

In [ ]:
# takes a lot of time
elevation = df[["LATITUDE", "LONGITUDE"]].apply(elevation_function, axis=1)

In [21]:
elevation = pd.DataFrame(elevation, columns={"ELEVATION"})
elevation

,ELEVATION
0,924.73
1,1831.26
2,1048.65
3,2361.86
4,2309.84
...,...
1880460,214.46
1880461,18.29
1880462,18.29
1880463,39.29


In [22]:
# save to use later
elevation.to_csv("elevation.csv")